In [243]:
from re import match

with open('input.txt') as f:
    file = f.read()

sensor_beacon_lst = []
for l in file.split('\n'):
    # using re.match
    res = match(r'Sensor at x=([\d-]+), y=([\d-]+): closest beacon is at x=([\d-]+), y=([\d-]+)', l)
    s_x, s_y, b_x, b_y = res.groups()
    sensor_beacon_lst.append(((int(s_x), int(s_y)), (int(b_x), int(b_y))))

In [244]:
def find_spread(sensor_beacon, row_index):
    # sensor beacon in the form of ((s_x, s_y), (b_x, b_y))
    # unpack here
    ((s_x, s_y), (b_x, b_y)) = sensor_beacon
    side_len = abs(s_x - b_x) + abs(s_y - b_y)
    if (row_index < s_y - side_len) or (row_index > s_y + side_len):
        return None
    half_len = side_len - abs(row_index - s_y)
    return (s_x - half_len, s_x + half_len)

# spread at specific row, e.g. row 10
total_spread = []

for sensor_beacon in sensor_beacon_lst:
    res = find_spread(sensor_beacon, 2000000)
    if res: total_spread.append(res)

In [245]:
total_spread

[(2551426, 2816974),
 (-529240, 2535964),
 (1936984, 2348326),
 (2775464, 2816974),
 (2907614, 4874050),
 (2816974, 2840710),
 (2634594, 2816974),
 (2964982, 4183712),
 (2816974, 2939030),
 (465023, 625803),
 (472610, 727652),
 (2121872, 2816974)]

In [246]:
# def find_island(total_spread):

#     while True:
#         ranges = []
#         prev_spread = total_spread.copy()
#         for x_start, x_end in total_spread:
#             if not ranges: ranges.append([x_start, x_end])
#             else:
#                 for i, (rx_start, rx_end) in enumerate(ranges):
#                     if x_start >= rx_start and x_end <= rx_end:
#                         break
#                     elif (x_start < rx_start and x_end < rx_start) or (x_start > rx_end and x_end > rx_end):
#                         ranges.append([x_start, x_end])
#                         break
#                     else:
#                         if x_start < rx_start:
#                             ranges[i][0] = x_start
#                         if x_end > rx_end:
#                             ranges[i][1] = x_end
#         if ranges == prev_spread:
#             break
#         total_spread = ranges.copy()
        
#     return ranges

# ranges = find_island(total_spread)

In [247]:
sorted(total_spread)

[(-529240, 2535964),
 (465023, 625803),
 (472610, 727652),
 (1936984, 2348326),
 (2121872, 2816974),
 (2551426, 2816974),
 (2634594, 2816974),
 (2775464, 2816974),
 (2816974, 2840710),
 (2816974, 2939030),
 (2907614, 4874050),
 (2964982, 4183712)]

In [248]:
def check_island(x_start, x_end, ranges):
    t_ranges = ranges.copy()
    if not ranges:
        t_ranges.append([x_start, x_end])
        return t_ranges
    j = 0
    for i, (rx_start, rx_end) in enumerate(t_ranges):
        if x_start > rx_end:
            j += 1
        # these 2 conditions checks for partial ovelap, 
        elif (rx_end >= x_start >= rx_start) or (rx_end >= x_end >= rx_start):
            if x_start < rx_start:
                t_ranges[i][0] = x_start
            if x_end > rx_end:
                t_ranges[i][1] = x_end
            break
        # this condition checks for full overlap, basically the inverse of the first if statement
        elif rx_start >= x_start and rx_end <= x_end:
            t_ranges[i] = [x_start, x_end]
            break
    if j == len(t_ranges):
        t_ranges.append([x_start, x_end])
    return t_ranges

def find_island(total_spread):
    total_spread = sorted(total_spread)
    ranges = []
    for x_start, x_end in total_spread:
        ranges = check_island(x_start, x_end, ranges)
    return ranges

ranges = find_island(sorted(total_spread))
ranges

[[-529240, 4874050]]

In [249]:
def count_ranges(ranges, row_num, sensor_beacon_lst=sensor_beacon_lst):
    num_hash = 0
    # add up all the ranges
    for x_start, x_end in ranges:
        num_hash += x_end - x_start + 1
    # remove instances where sensors or beacons are in the range
    # need to convert beacon into a set to remove duplciates

    for (s_x, s_y), (b_x, b_y) in sensor_beacon_lst:
        if s_y == row_num: num_hash -= 1
        # need to convert beacon to set first
        # if b_y == row_num: num_hash -= 1
    beacon_set = set([x[1] for x in sensor_beacon_lst])
    for (b_x, b_y) in beacon_set:
        if b_y == row_num: num_hash -= 1
    
    return num_hash

count_ranges(ranges, 2000000)

5403290

In [250]:
# part 1 summary
def get_row_range(row_num):
    total_spread = []

    for sensor_beacon in sensor_beacon_lst:
        res = find_spread(sensor_beacon, row_num)
        if res: total_spread.append(res)

    ranges = find_island(total_spread)
    count = count_ranges(ranges, row_num)

    return count

get_row_range(2000000)

5403290

In [251]:
# make map for part 2
from collections import defaultdict
sb_dict = defaultdict(list)

for (s_x, s_y), (b_x, b_y) in sensor_beacon_lst:
    sb_dict[s_y] = [(s_x, s_y)]
beacon_set = set([x[1] for x in sensor_beacon_lst])
for b_x, b_y in beacon_set:
    if b_y not in sb_dict:
        sb_dict[b_y] = [(b_x, b_y)]
    else: sb_dict[b_y].append((b_x, b_y))

In [252]:
sb_dict

defaultdict(list,
            {3759110: [(2557568, 3759110)],
             1861612: [(2684200, 1861612)],
             1946094: [(1003362, 1946094)],
             1481541: [(2142655, 1481541)],
             1955744: [(2796219, 1955744)],
             1818644: [(3890832, 1818644)],
             1921726: [(2828842, 1921726)],
             583957: [(2065227, 583957)],
             2088998: [(2725784, 2088998)],
             927734: [(3574347, 927734)],
             2652370: [(2939312, 2652370)],
             3681541: [(2495187, 3681541)],
             2054681: [(2878002, 2054681)],
             3235516: [(1539310, 3235516)],
             533006: [(545413, 533006)],
             3980292: [(1828899, 3980292)],
             2937931: [(3275729, 2937931)],
             3861189: [(600131, 3861189)],
             28975: [(2089895, 28975)],
             3942666: [(2960402, 3942666)],
             3905392: [(3785083, 3905392)],
             1077173: [(1721938, 1077173)],
             3751221: [(25

In [253]:
# part 2

def count_ranges2(ranges, row_num, limit, sensor_beacon_lst=sensor_beacon_lst):
    num_hash = 0
    # add up all the ranges
    for x_start, x_end in ranges:
        if x_end > limit: x_end = limit
        if x_start < 0: x_start = 0
        num_hash += x_end - x_start + 1
    # if num_hash != limit + 1:
    #     if row_num in sb_dict: num_hash += sb_dict[row_num]
    return num_hash

# part 2 summary
def get_row_range2(row_num, limit=20):
    total_spread = []

    for sensor_beacon in sensor_beacon_lst:
        res = find_spread(sensor_beacon, row_num)
        if res: total_spread.append(res)
    if row_num in sb_dict:
        total_spread.extend(sb_dict[row_num])
    # need to sort
    ranges = find_island(sorted(total_spread))
    count = count_ranges2(ranges, row_num, limit)

    return count, ranges

In [254]:
for i in range(4000000 + 1):
    count_hash, range_hash = get_row_range2(i, limit=4000000)
    if count_hash != (4000000 + 1):
        print(f'y={i}, x={range_hash}, count={count_hash}')
        break

y=2906626, x=[[-434016, 2572894], [2572896, 4135836]], count=4000000


In [258]:
(2572895 * 4000000) + 2906626

10291582906626